In [1]:
import joblib
import pandas as pd
import requests
from datetime import datetime

model = joblib.load("../Traffic_Analyzer/traffic_model.pkl")
le_day = joblib.load("../Traffic_Analyzer/day_encoder.pkl")
le_target = joblib.load("../Traffic_Analyzer/target_encoder.pkl")


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY = os.getenv("API_KEY")


In [3]:
coords = [
    (52.1548439, 4.4721623), (52.1683388, 4.4860599), (52.1476338, 4.4752352),
    (52.1686146, 4.4931477), (52.1519037, 4.5206045), (52.1632703, 4.4582134),
    (52.1417215, 4.4768956), (52.156569, 4.4950595), (52.1635883, 4.4580728),
    (52.1597215, 4.4968029), (52.1549858, 4.4949476), (52.1681008, 4.4864105),
    (52.1521453, 4.5208762), (52.1544488, 4.440454)
]

In [4]:
def estimate_car_count(current_speed, free_speed):
    if free_speed == 0:
        return 0
    ratio = current_speed / free_speed
    if ratio >= 0.9:
        return 40  # Low traffic
    elif ratio >= 0.6:
        return 90  # Medium
    elif ratio >= 0.3:
        return 140  # High
    else:
        return 180  # Very high


In [5]:
now = datetime.now()
hour = now.hour
day_encoded = le_day.transform([now.strftime("%A")])[0]
url_template = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"


for lat, lon in coords:
    params = {"key": API_KEY, "point": f"{lat},{lon}"}
    r = requests.get(url_template, params=params)
    data = r.json()
    
    speed = data["flowSegmentData"]["currentSpeed"]
    free = data["flowSegmentData"]["freeFlowSpeed"]

    car_count = estimate_car_count(speed, free)
    
    # Dummy values for other vehicles
    X_input = pd.DataFrame([[
        hour, day_encoded, car_count, 10, 5, 5
    ]], columns=["Hour", "DayEncoded", "CarCount", "BikeCount", "BusCount", "TruckCount"])
    
    label = model.predict(X_input)[0]
    situation = le_target.inverse_transform([label])[0]

    print(f"{lat},{lon} → {speed}/{free} km/h → Estimated traffic: {situation}")


52.1548439,4.4721623 → 44/44 km/h → Estimated traffic: low
52.1683388,4.4860599 → 31/31 km/h → Estimated traffic: low
52.1476338,4.4752352 → 44/44 km/h → Estimated traffic: low
52.1686146,4.4931477 → 48/48 km/h → Estimated traffic: low
52.1519037,4.5206045 → 42/49 km/h → Estimated traffic: normal
52.1632703,4.4582134 → 36/36 km/h → Estimated traffic: low
52.1417215,4.4768956 → 42/42 km/h → Estimated traffic: low
52.156569,4.4950595 → 24/30 km/h → Estimated traffic: normal
52.1635883,4.4580728 → 39/39 km/h → Estimated traffic: low
52.1597215,4.4968029 → 36/36 km/h → Estimated traffic: low
52.1549858,4.4949476 → 24/30 km/h → Estimated traffic: normal
52.1681008,4.4864105 → 46/46 km/h → Estimated traffic: low
52.1521453,4.5208762 → 23/31 km/h → Estimated traffic: normal
52.1544488,4.440454 → 85/85 km/h → Estimated traffic: low
